In [140]:
import numpy as np
import sys

from scipy.linalg import eig
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigs
from numpy import allclose

path = 'D:/repositories/composipy'

sys.path.append(path)

from time import time

from composipy import OrthotropicMaterial, LaminateProperty
from composipy import PlateStructure



In [141]:
#Properties (N and mm)
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21

# Plate Parameters
a = 360
b = 360
m = 10
n = 10


stacking = [45,-45,0,90,45,-45,0,90,45,-45]
stacking += stacking[::-1]

a = 360.
b = 360.

In [142]:
ply_1 = OrthotropicMaterial(E1, E2, v12, G12, t)
l1 = LaminateProperty(stacking, ply_1)
constraints={
                'x0': ['TX', 'TY', 'TZ'],
                'xa': ['TX', 'TY', 'TZ'],
                'y0': ['TX', 'TY', 'TZ'],
                'yb': ['TX', 'TY', 'TZ'],
            }
panel = PlateStructure(l1, a, b, m=m, n=n, Nxx=-1, Nyy=-1, constraints=constraints)

ti = time()
print('calculating K and KG')
K_KG = panel.calc_K_KG()
K = K_KG['K']
KG = K_KG['KG']
print(f'calculation done in {time()-ti} seconds')
print('calculating panel eig value')
ti = time()
print(panel.buckling_analysis())
print(f'calculation done in {time()-ti} seconds')

calculating K and KG
calculation done in 2.4665584564208984 seconds
calculating panel eig value
48.52457849227318
calculation done in 2.4968631267547607 seconds


In [149]:
eigvals, eigvecs = lb(K, KG, silent=False)
eigvals

Running linear buckling analysis...
		Eigenvalue solver... 
			eigsh() solver...
			finished!
		finished!
	first 5 eigenvalues:
		48.52457849227213
		115.33986497435149
		115.60711330382483
		194.09686865625642
		221.57771298566516


array([ 48.52457849, 115.33986497, 115.6071133 , 194.09686866,
       221.57771299, 222.15591277, 308.86123656, 309.48317985,
       368.99999417, 370.08445037, 436.71118574, 461.47246526,
       462.33970189, 561.6366027 , 563.3834036 , 599.62788016,
       600.77815938, 656.8345812 , 658.36588959, 776.37052798,
       801.3471514 , 802.96144218, 803.91915658, 804.12792152,
       898.07786955])

## comparison with compmech

In [144]:
from compmech.panel.panel import Panel
from compmech.analysis import lb, static

In [145]:
# skin panels
laminaprop = (E1, E2, v12, G12, G12, G12)
p1 = Panel(group='skin', Nxx=-1, Nyy=-1, x0=0, y0=0., a=a, b=b,m=12, n=12, plyt=t, stack=stacking, laminaprop=laminaprop)

In [146]:
p1.u1tx = 0 ; p1.u1rx = 0 ; p1.u2tx = 0 ; p1.u2rx = 0
p1.v1tx = 0 ; p1.v1rx = 0 ; p1.v2tx = 0 ; p1.v2rx = 0
p1.w1tx = 0 ; p1.w1rx = 1 ; p1.w2tx = 0 ; p1.w2rx = 1
p1.u1ty = 0 ; p1.u1ry = 0 ; p1.u2ty = 0 ; p1.u2ry = 0
p1.v1ty = 0 ; p1.v1ry = 0 ; p1.v2ty = 0 ; p1.v2ry = 0
p1.w1ty = 0 ; p1.w1ry = 1 ; p1.w2ty = 0 ; p1.w2ry = 1


In [147]:
k0_1 = p1.calc_k0()
kG0_1 = p1.calc_kG0()

eigvals, eigvecs = lb(k0_1, kG0_1, silent=True)
eigvals

		Calculating k0... 
		finished!
		Calculating kG0... 
		finished!


array([ 48.52457849, 115.33986497, 115.6071133 , 194.09686866,
       221.57771299, 222.15591277, 308.86123656, 309.48317985,
       368.99999417, 370.08445037, 436.71118574, 461.47246526,
       462.33970189, 561.6366027 , 563.3834036 , 599.62788016,
       600.77815939, 656.8345812 , 658.36588959, 776.37052798,
       801.34715129, 802.96144214, 803.91915658, 804.12792151,
       898.07786953])

In [148]:
k0_1

<432x432 sparse matrix of type '<class 'numpy.float64'>'
	with 186624 stored elements in Compressed Sparse Row format>